In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
from itertools import chain, cycle
import numpy as np
import pandas as pd
import random


class Clue:
    suspects = [
        "Miss Scarlet",
        "Mr. Green",
        "Mrs. White",
        "Mrs. Peacock",
        "Colonel Mustard",
        "Professor Plum",
        # Additional Master Detective Suspects
        "Miss Peach",
        "Sgt. Gray",
        "Monsieur Brunette",
        "Madame Rose",
    ]

    weapons = [
        "Candlestick",
        "Knife",
        "Lead Pipe",
        "Revolver",
        "Rope",
        "Wrench",
        # Additional Master Detective Weapons
        "Horseshoe",
        "Poison",
    ]

    rooms = [
        "Hall",
        "Lounge",
        "Dining Room",
        "Kitchen",
        "Ballroom",
        "Conservatory",
        "Billiard Room",
        "Library",
        "Study",
        # Additional Master Detective Rooms
        "Carriage House",
        "Cloak Room",
        "Trophy Room",
        "Drawing Room",
        "Gazebo",
        "Courtyard",
        "Fountain",
        "Studio",
    ]

    motives = [
        "Revenge",
        "Jealousy",
        "Greed",
        "Blackmail",
        "Power",
        "Cover-up",
        "Betrayal",
        "Obsession",
        "Inheritance",
        "Self-preservation",
    ]

    @staticmethod
    def get_times(start: str, end: str, freq: str) -> list:
        times = (
            (
                pd.date_range(start=start, end="23:59", freq=freq).time.tolist()
                + pd.date_range(start="00:00", end=end, freq=freq).time.tolist()
            )
            if end < start
            else pd.date_range(start=start, end=end, freq=freq).time.tolist()
        )
        return [time.strftime("%I:%M %p") for time in times]

In [ ]:
num_players = 6
elements = {
    "suspect": Clue.suspects[:6],
    "weapon": Clue.weapons[:6],
    "room": Clue.rooms[:6],
    # "motive": Clue.motives[:6],
    # "time": Clue.get_times("21:00", "03:00", "1h"),
}
solution = {element: random.choice(values) for element, values in elements.items()}

unfiltered_deck = list(chain(*elements.values()))
deck = unfiltered_deck.copy()
for cards in solution.values():
    deck.remove(cards)
random.shuffle(deck)
hands = [set(deck[i::num_players]) for i in range(num_players)]
hands.reverse()  # Reverse hands so players with fewer cards go first

for player, hand in enumerate(hands):
    print(f"Player {player + 1}'s Hand: {hand}")

print(f"\nSolution: {solution}")

history = []

index = {card: i for i, card in enumerate(unfiltered_deck)}

# Create a ground truth grid
ground_truth = np.zeros((len(unfiltered_deck), num_players))

# Fill in the ground truth grid based on the hands
for player, hand in enumerate(hands):
    for card in hand:
        ground_truth[index[card], player] = 1


def display_grid(grid: np.ndarray) -> None:
    df = pd.DataFrame(
        grid,
        columns=[f"{i + 1}" for i in range(num_players)],
    ).replace({np.nan: "", 0: "✗", 1: "✓"})
    df.index = pd.MultiIndex.from_tuples(
        [
            (element.capitalize(), card)
            for element in elements
            for card in elements[element]
        ],
        names=["Element", "Card"],
    )
    df.columns.name = "Player"
    print(df)


print("Ground Truth Grid:")
display_grid(ground_truth)

all_but = 0
suggestion_elimination = 0
element_deduction = 0
element_inference = 0
turn = 1

for player in cycle(range(num_players)):
    grid = np.full((len(unfiltered_deck), num_players), np.nan)
    last_grid = grid.copy()

    for card in hands[player]:
        grid[index[card], player] = 1.0
    for j, (suggestion, responses) in enumerate(history):
        j %= num_players
        for k, card in responses.items():
            # if player k did not reveal a card
            if card is None:
                # then all suggested cards must not be in player k's hand
                for suggested_card in suggestion:
                    grid[index[suggested_card], k] = 0.0
            # alternatively if player k revealed a card and we are the player who made the suggestion
            elif player == j:
                # then we know that player k has the revealed card
                grid[index[card], k] = 1.0

    while not np.array_equal(grid, last_grid, equal_nan=True):
        last_grid = grid.copy()
        for j in range(len(unfiltered_deck)):
            # if we have deduced that someone has a card
            if np.nansum(grid[j, :]) == 1:
                # then all other players must not have that card
                grid[j, np.isnan(grid[j, :])] = 0.0
        for j in range(num_players):
            # if we have deduced all the cards in player j's hand
            if np.nansum(grid[:, j]) == len(hands[j]):
                # then all other cards must not be in player j's hand
                grid[np.isnan(grid[:, j]), j] = 0.0
            # alternatively if we have deduced that all but len(hands[j]) cards are not in player j's hand
            elif (grid[:, j] == 0.0).sum() == len(unfiltered_deck) - len(hands[j]):
                # then the remaining cards must be in player j's hand
                grid[np.isnan(grid[:, j]), j] = 1.0
                all_but += 1
        for suggestion, responses in history:
            for k, card in responses.items():
                if (
                    # if player k revealed a card
                    card is not None
                    # and we don't know that any of the suggested cards are in player k's hand
                    and np.nansum(grid[[index[c] for c in suggestion], k]) < 1
                    # but we do know that all but one of the suggested cards are not in player k's hand
                    and (grid[[index[c] for c in suggestion], k] == 0.0).sum()
                    == len(suggestion) - 1
                ):
                    # then the one card that we are unsure of must be in player k's hand
                    grid[
                        index[
                            next(c for c in suggestion if np.isnan(grid[index[c], k]))
                        ],
                        k,
                    ] = 1.0
                    suggestion_elimination += 1
        start = 0
        for cards in elements.values():
            end = start + len(cards)
            element_grid = grid[start:end]
            if (
                # If we are unsure of any cells in the grid for a given game element
                np.isnan(element_grid).sum() > 0
                # but we know where all but one of the cards are
                and np.nansum(element_grid) == len(cards) - 1
            ):
                # then we can fill the remaining unknown cells with 0
                element_grid[np.isnan(element_grid)] = 0
                element_deduction += 1
            if (
                # If we know which card is part of the solution
                np.where(element_grid.sum(axis=0) == 0)[0].size == 1
                # and there are any cards with one unknown cell
                and np.where(np.isnan(element_grid).sum(axis=0) == 1)[0].size > 0
            ):
                # then we can fill the unknown cells with 1
                solvable_rows = element_grid[
                    np.where(np.isnan(element_grid).sum(axis=0) == 1)
                ]
                solvable_rows[np.isnan(solvable_rows)] = 1
                element_inference += 1
            start += len(cards)

    print(f"Player {player + 1} Grid:")
    display_grid(grid)
    np.testing.assert_array_equal(
        grid[~np.isnan(grid)],
        ground_truth[~np.isnan(grid)],
        err_msg="Non-NaN values in grid do not match ground truth",
    )

    accusation = len(np.where(grid.sum(axis=1) == 0)[0]) == len(elements)
    suggestion = []
    start = 0
    for element, cards in elements.items():
        end = start + len(cards)
        if not accusation and random.random() < 0.4 and np.sum(grid[start:end, player]) > 0:
            suggestion.append(
                unfiltered_deck[
                    np.random.choice(np.where(grid[start:end, player] == 1)[0] + start)
                ]
            )
        else:
            suggestion.append(
                unfiltered_deck[
                    np.random.choice(
                        np.where(
                            (np.sum if accusation else np.nansum)(grid, axis=1)[
                                start:end
                            ]
                            == 0
                        )[0]
                        + start
                    )
                ]
            )
        start += len(cards)

    if accusation:
        print(suggestion)
        print(solution)
        assert all(
            suggestion[i] == solution[element] for i, element in enumerate(elements)
        )
        print(f"Player {player + 1} won on turn {turn}!")
        print(f"# of all but deductions: {all_but}")
        print(f"# of suggestion eliminations: {suggestion_elimination}")
        print(f"# of element deductions: {element_deduction}")
        print(f"# of element inferences: {element_inference}")
        break

    print(f"Player {player + 1} suggests: {suggestion}")

    responses = {}
    for j in chain(range(player + 1, num_players), range(player)):
        responses[j] = None
        suggestion_copy = suggestion.copy()
        random.shuffle(suggestion_copy)
        for card in suggestion_copy:
            if card in hands[j]:
                responses[j] = card
                break
        if responses[j] is not None:
            break
    history.append((suggestion, responses))
    turn += 1

In [ ]:
from itertools import chain, cycle
import numpy as np
from ortools.sat.python import cp_model
import pandas as pd
import random


num_players = 6
elements = {
    "suspect": Clue.suspects[:10],
    "weapon": Clue.weapons[:12],
    "room": Clue.rooms[:8],
    "motive": Clue.motives[:10],
    "time": Clue.get_times("21:00", "03:00", "1h"),
}
solution = {element: random.choice(values) for element, values in elements.items()}

unfiltered_deck = list(chain(*elements.values()))
deck = unfiltered_deck.copy()
for cards in solution.values():
    deck.remove(cards)
random.shuffle(deck)
hands = [set(deck[i::num_players]) for i in range(num_players)]
hands.reverse()  # Reverse hands so players with fewer cards go first

for player, hand in enumerate(hands):
    print(f"Player {player + 1}: {hand}")

print(f"\nSolution: {solution}")

history = []

index = {card: i for i, card in enumerate(unfiltered_deck)}

# Create a ground truth grid
ground_truth = np.zeros((len(unfiltered_deck), num_players))

# Fill in the ground truth grid based on the hands
for player, hand in enumerate(hands):
    for card in hand:
        ground_truth[index[card], player] = 1


def display_grid(grid: np.ndarray) -> None:
    df = pd.DataFrame(
        grid,
        columns=[f"{i + 1}" for i in range(num_players)],
    ).replace({np.nan: "", 0: "✗", 1: "✓"})
    df.index = pd.MultiIndex.from_tuples(
        [
            (element.capitalize(), card)
            for element in elements
            for card in elements[element]
        ],
        names=["Element", "Card"],
    )
    df.columns.name = "Player"
    print(df)


print("Ground Truth Grid:")
display_grid(ground_truth)

all_but = 0
suggestion_elimination = 0
element_deduction = 0
element_inference = 0
turn = 1

model = cp_model.CpModel()

vars = [
    [model.new_bool_var(f"{j},{k}") for k in range(num_players)]
    for j in range(len(unfiltered_deck))
]

# Enforce that each card i is assigned to at most one player j
for j in range(len(unfiltered_deck)):
    model.add(sum(vars[j]) <= 1)

# Enforce that each player j has exactly len(hands[j]) cards assigned to them
for j, hand in enumerate(hands):
    # Create a list of Boolean indicators where vars[card] == j
    assigned_to_j = [row[j] for row in vars]
    # Sum these indicators using LinearExpr.Sum and set it equal to the number of cards in hand j
    model.add(sum(assigned_to_j) == len(hand))

# Enforce that there are len(elements[element]) - 1 cards assigned to players for each element
start = 0
for cards in elements.values():
    end = start + len(cards)
    # Create a list of Boolean indicators where vars[card] == j
    assigned_to_j = [var for row in vars[start:end] for var in row]
    # Sum these indicators using LinearExpr.Sum and set it equal to the number of cards in hand j
    model.add(sum(assigned_to_j) == len(cards) - 1)
    start = end

solver = cp_model.CpSolver()
solver.parameters.enumerate_all_solutions = True
solver.parameters.max_time_in_seconds = 1


class SolutionCallback(cp_model.CpSolverSolutionCallback):
    def __init__(self) -> None:
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.grid = np.zeros((len(unfiltered_deck), num_players))
        self.num_solutions = 0

    def on_solution_callback(self):
        self.grid += np.array([[self.value(var) for var in row] for row in vars])
        self.num_solutions += 1


for player in cycle(range(num_players)):
    # Add assumptions for the cards in player i's hand
    for card in hands[player]:
        model.add_assumption(vars[index[card]][player])

    # Add assumptions for the cards that were revealed to player i in previous turns
    for j, (_, responses) in enumerate(history):
        j %= num_players
        if player == j:
            for k, card in responses.items():
                if card is not None:
                    model.add_assumption(vars[index[card]][k])

    callback = SolutionCallback()
    status = solver.solve(model, callback)
    assert status == cp_model.OPTIMAL or status == cp_model.FEASIBLE
    model.clear_assumptions()
    grid = callback.grid / callback.num_solutions
    # set every cell that does not equal zero or one to NaN
    grid[(grid != 0) & (grid != 1)] = np.nan
    print(f"Player {player + 1} Grid:")
    display_grid(grid)
    np.testing.assert_array_equal(
        grid[~np.isnan(grid)],
        ground_truth[~np.isnan(grid)],
        err_msg="Non-NaN values in grid do not match ground truth",
    )

    accusation = len(np.where(grid.sum(axis=1) == 0)[0]) == len(elements)
    suggestion = []
    start = 0
    for element, cards in elements.items():
        end = start + len(cards)
        if not accusation and random.random() < 0.4 and np.sum(grid[start:end, player]) > 0:
            suggestion.append(
                unfiltered_deck[
                    np.random.choice(np.where(grid[start:end, player] == 1)[0] + start)
                ]
            )
        else:
            suggestion.append(
                unfiltered_deck[
                    np.random.choice(
                        np.where(
                            (np.sum if accusation else np.nansum)(grid, axis=1)[
                                start:end
                            ]
                            == 0
                        )[0]
                        + start
                    )
                ]
            )
        start += len(cards)

    if accusation:
        print(suggestion)
        print(solution)
        assert all(
            suggestion[i] == solution[element] for i, element in enumerate(elements)
        )
        print(f"Player {player + 1} won on turn {turn}!")
        break

    print(f"Player {player + 1} suggests: {suggestion}")

    responses = {}
    for j in chain(range(player + 1, num_players), range(player)):
        responses[j] = None
        suggestion_copy = suggestion.copy()
        random.shuffle(suggestion_copy)
        for card in suggestion_copy:
            if card in hands[j]:
                responses[j] = card
                # Everyone knows that player j has at least one of the suggested cards
                model.add_at_least_one([vars[index[card]][j] for card in suggestion])
                break
        if responses[j] is None:
            # Everyone knows that player j does not have any of the suggested cards
            model.add(sum(vars[index[card]][j] for card in suggestion) == 0)
        else:
            break
    history.append((suggestion, responses))
    turn += 1

In [7]:
from lib.clue import Clue

game = Clue()
game.play()

Player 1's Hand: {'Miss Scarlet', 'Dining Room'}
Player 2's Hand: {'Hall', 'Knife'}
Player 3's Hand: {'Mrs. White', 'Lead Pipe'}
Solution: {'suspect': 'Mr. Green', 'weapon': 'Candlestick', 'room': 'Lounge'}
Player                1  2  3
Element Card                 
Suspect Miss Scarlet  ✓  ✗  ✗
        Mr. Green     ✗  ✗  ✗
        Mrs. White    ✗  ✗  ✓
Weapon  Candlestick   ✗  ✗  ✗
        Knife         ✗  ✓  ✗
        Lead Pipe     ✗  ✗  ✓
Room    Hall          ✗  ✓  ✗
        Lounge        ✗  ✗  ✗
        Dining Room   ✓  ✗  ✗
Player 1's Simple Solver Grid:
Player                1  2  3
Element Card                 
Suspect Miss Scarlet  ✓  ✗  ✗
        Mr. Green     ✗      
        Mrs. White    ✗      
Weapon  Candlestick   ✗      
        Knife         ✗      
        Lead Pipe     ✗      
Room    Hall          ✗      
        Lounge        ✗      
        Dining Room   ✓  ✗  ✗
Player 1's CP-SAT Solver Grid:
Player                1  2  3
Element Card                 
Suspect Mis

AssertionError: Simple Solver and CP-SAT Solver grids do not match

In [9]:
import numpy as np
from typing import Optional, Union

player = 1
grid = np.full((len(game.index), game.num_players + 1), np.nan)
last_grid = grid.copy()
constraints: list[
    tuple[
        Union[np.ndarray, tuple[np.ndarray, ...]],
        Union[int, tuple[Optional[int], Optional[int]]],
    ]
] = []

# Each card may only be in one place
for i in range(len(game.index)):
    constraints.append((grid[i], 1))

# Each player has game.hands[i] cards
for i, hand in enumerate(game.hands):
    constraints.append((grid[:, i], len(hand)))

# The solution must have exactly one card from each game element
start = 0
for cards in game.elements.values():
    end = start + len(cards)
    constraints.append((grid[start:end, game.num_players], 1))
    start = end

# Fill in the grid with the known cards
for card in game.hands[player]:
    grid[game.index[card], player] = 1

one_of: dict[int, list[set[int]]] = {i: [] for i in range(game.num_players)}

# Fill in the grid with the known cards from previous turns
for suggesting_player, (suggestion, responses) in enumerate(game.history):
    suggesting_player %= game.num_players
    for responding_player, card in responses.items():
        if card is not None:
            if player == suggesting_player:
                grid[game.index[card], responding_player] = 1
            elif player != responding_player:
                indices = [game.index[c] for c in suggestion]
                # At least one of the suggested cards is in the responding player's hand
                constraints.append(
                    (
                        tuple(grid[i : i + 1, responding_player] for i in indices),
                        (1, len(suggestion)),
                    )
                )
                # And no more than len(game.hands[responding_player]) - 1 of the unsuggested cards are in the responding player's hand
                constraints.append(
                    (
                        tuple(
                            grid[i + 1 : j, responding_player]
                            for i, j in zip([-1] + indices, indices + [len(game.index)])
                        ),
                        (0, len(game.hands[responding_player]) - 1),
                    )
                )
                for previous_indices in one_of[responding_player]:
                    if previous_indices.isdisjoint(indices):
                        union = sorted(previous_indices.union(indices))
                        constraints.append(
                            (
                                tuple(
                                    grid[i + 1 : j, responding_player]
                                    for i, j in zip(
                                        [-1] + union, union + [len(game.index)]
                                    )
                                ),
                                (0, len(game.hands[responding_player]) - len(union) // len(indices)),
                            )
                        )
                        one_of[responding_player].append(set(union))
                one_of[responding_player].append(set(indices))
                print(
                    f"Player {responding_player + 1} has at least one of {suggestion}"
                )
        else:
            for card in suggestion:
                grid[game.index[card], responding_player] = 0

while not np.array_equal(grid, last_grid, equal_nan=True):
    last_grid = grid.copy()
    for views, bounds in constraints:
        if not isinstance(views, tuple):
            views = (views,)
        if isinstance(bounds, int):
            lower_bound = upper_bound = bounds
        else:
            lower_bound, upper_bound = bounds
        values = np.concatenate([view.flatten() for view in views])
        if np.sum(np.nan_to_num(values, nan=1)) == lower_bound:
            for view in views:
                view[np.isnan(view)] = 1
        if np.nansum(values) == upper_bound:
            for view in views:
                view[np.isnan(view)] = 0

game.print_grid(grid[:, :-1])

Player 1 has at least one of ['Miss Scarlet', 'Lead Pipe', 'Dining Room']
Player 3 has at least one of ['Miss Scarlet', 'Lead Pipe', 'Dining Room']
Player 1 has at least one of ['Miss Scarlet', 'Knife', 'Lounge']
Player                1  2  3
Element Card                 
Suspect Miss Scarlet     ✗  ✗
        Mr. Green        ✗  ✗
        Mrs. White    ✗  ✗  ✓
Weapon  Candlestick   ✗  ✗  ✗
        Knife         ✗  ✓  ✗
        Lead Pipe     ✗  ✗  ✓
Room    Hall          ✗  ✓  ✗
        Lounge           ✗  ✗
        Dining Room      ✗  ✗


In [17]:
one_of

{0: [], 1: [{1, 3, 7}, {1, 2, 3, 5, 7, 8}, {2, 5, 8}], 2: []}

In [ ]:
"""
Player 4's CP-SAT Solver Grid:
Player                   1  2  3  4
Element Card                       
Suspect Miss Scarlet        ✗     ✗
        Mr. Green        ✗  ✗  ✗  ✓
        Mrs. White       ✗  ✗     ✗
        Mrs. Peacock     ✗  ✓  ✗  ✗
        Colonel Mustard        ✗  ✗
        Professor Plum            ✗
Weapon  Candlestick      ✗  ✗     ✗
        Knife            ✗  ✗     ✗
        Lead Pipe        ✓  ✗  ✗  ✗
        Revolver                  ✗
        Rope                   ✗  ✗
        Wrench           ✗  ✗  ✗  ✓
Room    Hall                   ✗  ✗
        Lounge           ✗  ✗  ✗  ✓
        Dining Room      ✗  ✗     ✗
        Kitchen             ✗     ✗
        Ballroom         ✗  ✗  ✗  ✓
        Conservatory           ✗  ✗
"""

In [ ]:
from itertools import chain
import numpy as np
import sympy as sp

grid = np.array(
    [
        [
            sp.Symbol(f"{player} has {card}", integer=True)
            for player in chain(
                (f"Player {player + 1}" for player in range(game.num_players)),
                ["Solution"],
            )
        ]
        for card in game.index
    ]
)

constraints = []

for player in range(len(game.index)):
    constraints.append(grid[player].sum() == 1)

for player in range(game.num_players):
    constraints.append(grid[:, player].sum() == len(game.hands[player]))

for cards in game.elements.values():
    constraints.append(
        grid[[game.index[card] for card in cards], game.num_players].sum() == 1
    )

for player, (suggestion, responses) in enumerate(game.history):
    for j, card in responses.items():
        if card is not None:
            constraints.append(
                grid[[game.index[card] for card in suggestion], j].sum() >= 1
            )
        else:
            constraints.append(
                grid[[game.index[card] for card in suggestion], j].sum() == 0
            )

player = 0

for card in game.hands[player]:
    constraints.append(grid[game.index[card], player] == 1)

for player in range(len(game.index)):
    for j in range(game.num_players):
        symbol = grid[player, j]
        print(sp.solve(constraints + [symbol == 0], symbol))

# sp.linsolve(constraints, *grid.flatten())

In [ ]:
from functools import partial
import kanren as kr

grid = np.array(
    [
        [
            kr.var(f"{player} has {card}")
            for player in chain(
                (f"Player {player + 1}" for player in range(game.num_players)),
                ["Solution"],
            )
        ]
        for card in game.index
    ]
)

kr.run(
    0,
    grid[0, 0],
    kr.lall(*np.vectorize(partial(kr.membero, ls=(0, 1)))(grid).flatten())
)

In [ ]:
np.vectorize(kr.eq)(grid[0], 1)